In [ ]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [ ]:
from pathlib import Path

import requests


if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
import re

import davos
import IPython

from utils import mark, raises, run_tests

# tests for `davos.implementations` (`__init__.py`)
**Notes**: 
- tests for whether correct implementation functions are imported are in the respective modules' test notebooks
- `fget`/`fset` functions not tested here are tested in `test_config.ipynb`

In [ ]:
 def test_conda_avail_fset_raises():
    """check that `conda_avail` config field is read-only"""
    match = re.escape("'davos.config.conda_avail': field is read-only")
    with raises(davos.core.exceptions.DavosConfigError, match=match):
        davos.config.conda_avail = True

In [ ]:
@mark.colab
def test_conda_env_fset_raises_noconda():
    """
    check that trying to set the `conda_env` config field raises an 
    error if the `conda` executable is not available
    
    test restricted to colab, where conda is not available
    """
    match = re.escape(
        "'davos.config.conda_env': cannot set conda environment. No local "
        "conda installation found"
    )
    with raises(davos.core.exceptions.DavosConfigError, match=match):
        davos.config.conda_env = 'arbitrary-environment-name'

In [ ]:
@mark.jupyter
def test_conda_env_fset_raises_noenv():
    """
    check that trying to set the `conda_env` config field to an 
    environment that does not exist raises an error
    """
    local_envs = {"', '".join(davos.config.conda_envs_dirs.keys())}
    match = re.escape(
        "'davos.config.conda_env': unrecognized environment name: "
        f"'fake-env-name'. Local environments are:\n\t'{local_envs}'"
    )
    with raises(davos.core.exceptions.DavosConfigError, match=match):
        davos.config.conda_env = 'fake-env-name'

In [ ]:
 def test_conda_envs_dirs_fset_raises():
    """check that `conda_envs_dirs` config field is read-only"""
    match = re.escape("'davos.config.conda_envs_dirs': field is read-only")
    with raises(davos.core.exceptions.DavosConfigError, match=match):
        davos.config.conda_envs_dirs = {'fake-foo': 'fake/path/to/fake-foo'}

In [ ]:
run_tests()